**วิธีเเรกใช้ google/flan-t5-base ช่วย clean**

ไม่เเนะนำวิธีเนื่องจากเป็นโมเดลที่ออกมาค่อนข้างนานเเล้ว เเต่ที่ลอง clean ด้วยวิธีนี้จะได้เปรียบเทียบได้ง่ายๆ กับอีกวิธี

เชื่อม google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
# ✅ ติดตั้งไลบรารีที่จำเป็น
!pip install transformers accelerate sentencepiece --quiet
!pip install pytesseract pdf2image transformers accelerate \
    sentence-transformers faiss-cpu langchain \
    unstructured chromadb streamlit \
    torchvision torchaudio --quiet

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

# โหลดโมเดล (ไม่ต้องใช้ HF Token)
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=False)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name, use_auth_token=False).to("cuda" if torch.cuda.is_available() else "cpu")


โหลดข้อมูลจาก google drive ที่เราทำการรวมเข้าด้วยกัน

In [ ]:
# โหลดข้อความจาก Google Drive
file_path = "/content/drive/MyDrive/LockLearn/combined_text.txt"
with open(file_path, "r", encoding="utf-8") as f:
    raw_text = f.read()


In [ ]:
# แบ่งข้อความออกเป็นหน้า ๆ (เช่น ทุก ๆ 40 บรรทัด = 1 หน้า)
# ลอง
lines = raw_text.split("\n")
pages = ["\n".join(lines[i:i+40]) for i in range(0, len(lines), 40)]


In [ ]:
# ฟังก์ชันให้ LLM ช่วย clean หน้า
def clean_text_with_llm(text):
    prompt = f"Please correct any spacing errors, fix sentence breaks, and clean up OCR mistakes in the following text:\n{text}"
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=512)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [ ]:
# ประมวลผลทีละหน้า
cleaned_pages = []
for i, page in enumerate(pages):
    print(f"✨ Cleaning page {i+1}/{len(pages)}...")
    cleaned = clean_text_with_llm(page)
    cleaned_pages.append(cleaned)


รวมทุกหน้าเข้าด้วยกัน

In [ ]:
# รวมทุกหน้ากลับเป็นไฟล์เดียว
output_path = "/content/drive/MyDrive/LockLearn/cleaned_text.txt"
with open(output_path, "w", encoding="utf-8") as f:
    f.write("\n\n--- PAGE BREAK ---\n\n".join(cleaned_pages))

print("✅ เสร็จแล้ว! ไฟล์ถูกบันทึกที่:", output_path)